In [58]:
import pandas as pd
import numpy as np
# Dataset
df1 = pd.read_csv("bengaluru_house_prices.csv")
df1.head()
# Removing Extra Columns
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()
df2.shape
# Checking for any NA Values in Dataset (Overall 74 Rows Deleted)
df3 = df2.dropna()
df3.isnull().sum()
df3.shape
# Main Backend Calculation, bhk - Stores value of BHK
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True
# All the non-Float "total_sqft" Values are noted  
df3[~df3['total_sqft'].apply(is_float)].head()
# For "total_sqft" Values existing as Range -> we will be using the mid value
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None   
df4 = df3.copy()
# Replacing all the Range of "total_sqft" Values with the mid value
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head()
# A new Dataset with a new Column: Price per Sqft (Price in Lakhs)
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()
# Storing all this Data in a new Dataset (name: bhp.csv)
df5.to_csv("bhp.csv",index=False)
# location - Stores location of Plot
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
# Dimensionality Reduction: Any location having less than 10 value counts should be tagged as "Other" location. This way number of categories can be reduced by huge amount.
location_stats_less_than_10 = location_stats[location_stats<=10]
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
df5.head()
# Outlier Removal -> using Business Logic (for example: 1BHK-300Sqft but Some Records claim 2BHK-400Sqft(Suspicious))
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape
# Outlier Removal -> using Standard Deviation and Mean (min price per sqft is 267 rs/sqft whereas max is 1800000, this shows a wide variation in property prices)
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape
# Outlier Removal (Remove apartments where for same location, the price of (for example) 3 bhk apartment is less than 2 bhk apartment (with same sqft area))
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
df8.shape
# Outlier Removal (Total bathroom = Total bedroom + 1 max, Anything above that is a data error)
df9 = df8[df8.bath<df8.bhk+2]
df9.shape
# Dataset (df10) after removing all Suspicious Apartments
df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head()
# One Hot Encoding: to give Equal Weightage to each and every Location
dummies = pd.get_dummies(df10.location)
dummies.head()
df11 = pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()
# Final Dataset(df12) after removing all Suspicious Apartments and Concatenating Encoded locations with Dataset 
df12 = df11.drop('location',axis='columns')
df12.head()
# LinearRegression
x = df12.drop(['price'],axis='columns')
x.head()
y = df12.price
y.head()
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=10)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)
# Final Calculation to Predict the Price of Apartment
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(x.columns==location)[0][0]
    X = np.zeros(len(x.columns))
    X[0] = sqft
    X[1] = bath
    X[2] = bhk
    if loc_index >= 0:
        X[loc_index] = 1
    return model.predict([X])[0]
# Sample Input Data
predict_price('Whitefield',2100,3,3)
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(model,f)
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

/var/folders/4q/v9qh_6f93yg92kpj0zvyc2yc0000gn/T/ipykernel_99926/693229388.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
/Users/ayushgarg/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
